In [265]:
# numpy experimentation
a=np.ones((10))
b=np.zeros(10)
b[1]=1
print a.shape, " ", b.shape
c =1.2*np.outer(a,b)
print (a*b)
print np.square(c)
print c
print c[:,1].shape
a[0][b==0]=4
print a

(10,)   (10,)
[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
[[ 0.    1.44  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.44  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.44  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.44  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.44  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.44  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.44  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.44  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.44  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.44  0.    0.    0.    0.    0.    0.    0.    0.  ]]
[[ 0.   1.2  0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   1.2  0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   1.2  0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   1.2  0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   1.2  0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   1.2  0.   0.   0.   0.   0.   0.   0.   

TypeError: 'numpy.float64' object does not support item assignment

In [197]:
import numpy as np
import struct
import sys

def read_file(filename):
    with open(filename,'rb') as fp:
        zero, data_type, dims = struct.unpack('>HBB', fp.read(4))
        shape = tuple(struct.unpack('>I', fp.read(4))[0] for d in range(dims))
        np_array = np.frombuffer(fp.read(), dtype=np.uint8).reshape(shape)
    return np_array

def preprocess(image_file, label_file):
    images = read_file(image_file)
    labels = read_file(label_file)
    if (len(labels) > 10000):
        labels = labels[:10000]
        images = images[:10000]    
    images = images/255.0
    images = images.reshape( (10000, 784))

    labels = labels.reshape(-1,1)
    data = np.concatenate((images, labels), axis=1)
    np.random.shuffle(data)
    return data

def get_features_labels(data, bias):
    examples = data[:,:-1]
    labels = data[:,-1]
    classifier_labels = np.zeros((10, len(labels)))
    for i in range(10):
        classifier_labels[i][labels == i] = 1
    examples = np.append(examples, bias, 1)
    return examples, labels, classifier_labels

def get_true_label(digit, perceptron_type):
    if digit == perceptron_type:
        return 1
    return 0

In [206]:
def inference(test_data, weights):
    data_size = len(test_data)
    bias = np.ones((data_size,1))
    examples, labels, classifier_labels = get_features_labels(test_data, bias)
    prediction = np.ones(data_size, dtype = int)
    correct = 0
    for i, example in enumerate(examples):
        activation_values = sigmoid(np.sum(weights*example, axis = 1))
        prediction[i] = np.argmax(activation_values)
        if prediction[i] == labels[i]:
            correct += 1
    accuracy = correct*1.0/data_size
    return prediction, labels, accuracy

In [243]:
from numpy import linalg as LA
def sigmoid(value):
    value[value > 100] = 100
    value[value < -100] = -100
    return 1/(1+np.exp(-value))

In [66]:
path = "../hw2/DATA_FOLDER/"

train_data = preprocess(path + '/train-images.idx3-ubyte', path + '/train-labels.idx1-ubyte')
# print train_data[1]
test_data = preprocess(path + '/t10k-images.idx3-ubyte', path + '/t10k-labels.idx1-ubyte')



In [118]:
def propagate(w, X, Y):

    m = X.shape[1]

    A = sigmoid(np.dot(w,X.T)) 
    print A.shape

    cost = -1/m * np.sum(Y * np.log(A) + (1-Y) * (np.log(1-A)))

    dz= (1/m)*(A - Y)
    dw = np.dot(dz,X)
    print dw.shape

    cost = np.squeeze(cost)
    grads = {"dw": dw}

    return grads, cost

In [ ]:
def train_sgd(train_data, num_epoches, learning_rate, lamda, test_data):
    data_size = len(train_data)
    weights = np.random.uniform(-0.1,0.1,[10,785])
    bias = np.ones((data_size,1))
    for epoch in range(num_epoches):
        loss = np.zeros(10)
        np.random.shuffle(train_data)
        examples, labels, classifier_labels = get_features_labels(train_data, bias)
        delta_weights = np.zeros((10,785))
        for i,example in enumerate(examples):
            z = np.sum(weights*example, axis = 1)
            y_pred = sigmoid(z)
            delta_weights += learning_rate*np.outer(y_pred-classifier_labels[:,i],example)
            loss+=classifier_labels[:,i]*np.log(y_pred+0.00000001)+(1-classifier_labels[:,i])*np.log(1-y_pred+0.00000001)
        
        weights -= delta_weights - lamda*weights
        loss /= -data_size*1.0 + lamda
        loss = np.sum(loss)
        train_prediction, train_labels, train_accuracy = inference(train_data[:train_data_size], weights)
        test_prediction, test_labels, test_accuracy = inference(test_data, weights)
        print "epoch ", epoch, ": Training loss: ", loss, " Training Accuracy: ", train_accuracy, " Test Accuracy: ", test_accuracy
#         if loss < 3.4 and epoch > 50:
#             return weights, loss
    return weights, loss

In [302]:
def train_gd(train_data, num_epoches, learning_rate, lamda, test_data):
    data_size = len(train_data)
    weights = np.random.uniform(-0.1,0.1,[10,785])
    bias = np.ones((data_size,1))
    for epoch in range(num_epoches):
        loss = np.zeros(10)
        np.random.shuffle(train_data)
        examples, labels, classifier_labels = get_features_labels(train_data, bias)
        delta_weights = np.zeros((10,785))
        for i,example in enumerate(examples):
            z = np.sum(weights*example, axis = 1)
            y_pred = sigmoid(z)
            delta_weights += learning_rate*np.outer(y_pred-classifier_labels[:,i],example)
            loss+=classifier_labels[:,i]*np.log(y_pred+0.0000000001)+(1-classifier_labels[:,i])*np.log(1-y_pred+0.0000000001)
        
        weights = weights - delta_weights - lamda*weights
        loss = loss*-1.0/data_size + lamda*np.sum(np.square(weights), axis = 1)/2.0
        loss = np.sum(loss)/10.0
        train_prediction, train_labels, train_accuracy = inference(train_data[:train_data_size], weights)
        test_prediction, test_labels, test_accuracy = inference(test_data, weights)
        print "epoch ", epoch, ": Training loss: ", loss, " Training Accuracy: ", train_accuracy, " Test Accuracy: ", test_accuracy
#         if loss < 3.4 and epoch > 50:
#             return weights, loss
    return weights, loss 

In [ ]:
train_data_size = 10000
epochs = 200
learning_rate = 0.0001
lamda = 0.01
# lamda = 0
gd_weights, train_loss = train_gd(train_data[:10000], epochs, learning_rate, lamda, test_data)
test_prediction, test_labels, test_accuracy = inference(test_data, gd_weights)
print "test accuracy ", test_accuracy, " training loss: ", train_loss

epoch  0 : Training loss:  0.777608446606  Training Accuracy:  0.1909  Test Accuracy:  0.1958
epoch  1 : Training loss:  1.38404539488  Training Accuracy:  0.3439  Test Accuracy:  0.3567
epoch  2 : Training loss:  0.904091957421  Training Accuracy:  0.41  Test Accuracy:  0.4216
epoch  3 : Training loss:  0.476009712109  Training Accuracy:  0.5657  Test Accuracy:  0.5699
epoch  4 : Training loss:  0.257369494884  Training Accuracy:  0.6761  Test Accuracy:  0.6622
epoch  5 : Training loss:  0.228449768713  Training Accuracy:  0.8028  Test Accuracy:  0.7966
epoch  6 : Training loss:  0.167645122412  Training Accuracy:  0.7366  Test Accuracy:  0.7239
epoch  7 : Training loss:  0.171673585638  Training Accuracy:  0.7581  Test Accuracy:  0.7541
epoch  8 : Training loss:  0.176319548407  Training Accuracy:  0.718  Test Accuracy:  0.7018
epoch  9 : Training loss:  0.18101169798  Training Accuracy:  0.7723  Test Accuracy:  0.7689
epoch  10 : Training loss:  0.18389401357  Training Accuracy:  0.

epoch  87 : Training loss:  0.121891921726  Training Accuracy:  0.8856  Test Accuracy:  0.8773
epoch  88 : Training loss:  0.122900194412  Training Accuracy:  0.8894  Test Accuracy:  0.8782
epoch  89 : Training loss:  0.121750398164  Training Accuracy:  0.8859  Test Accuracy:  0.8777
epoch  90 : Training loss:  0.12276027662  Training Accuracy:  0.8899  Test Accuracy:  0.8783
epoch  91 : Training loss:  0.12161685132  Training Accuracy:  0.886  Test Accuracy:  0.8777
epoch  92 : Training loss:  0.122628170469  Training Accuracy:  0.8899  Test Accuracy:  0.8784
epoch  93 : Training loss:  0.121490730154  Training Accuracy:  0.8862  Test Accuracy:  0.8779
epoch  94 : Training loss:  0.122503343651  Training Accuracy:  0.89  Test Accuracy:  0.8783
epoch  95 : Training loss:  0.121371531034  Training Accuracy:  0.8864  Test Accuracy:  0.8785
epoch  96 : Training loss:  0.12238530908  Training Accuracy:  0.8901  Test Accuracy:  0.8782
epoch  97 : Training loss:  0.121258792603  Training Acc

In [71]:
a= np.zeros((10,1))
sigmoid(a)

array([[ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5]])